1. New_data EDA update
    *  Korea Province #patinet 데이터의 지역별 결측치가 많았는데, case data를 이용해서 EDA 가 가능해졌습니다.
    *  test, negative, confirmed analysis #time 데이터셋에서 역시 추가로 EDA 가능해서 업데이트합니다.
2. Intermational data EDA
3. Patient count EDA analysis plus Prophet time-series analysis.
4. patient released count EDA analysis.  Classification of age_generation, sex, etc
5. ARIMA model. SVM, Linear Regressor, MLPm LSTM,

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import date, timedelta

from fbprophet import Prophet
from fbprophet.plot import plot_plotly, add_changepoints_to_plot
from plotly.offline import iplot, init_notebook_mode
import plotly.graph_objs as go
import plotly.offline as py
import plotly.express as px

import seaborn as sns
import scipy as sp

In [ ]:
case = pd.read_csv('../input/coronavirusdataset/case.csv')
case

In [ ]:
region = case.groupby(['province','city']).confirmed.sum()
region =pd.DataFrame(region)
region

In [ ]:
count = region.groupby(['province']).sum() 
count = pd.DataFrame(count)
count

In [ ]:
count.reset_index(inplace=True)
count.index=["Busan","Chungcheongbuk-do","Chungcheongnam-do","Daegu","Daejeon","Gangwon-do","Gwangju","Gyeonggi-do","Gyeongsangbuk-do","Gyeongsangnam-do",
            "Incheon","Jeju-do","Jeollabuk-do","Jeollanam-do","Sejong","Seoul","Ulsan"]
fig = px.pie(count, values=count.confirmed, names=count.index)
fig.show()

# 전체테스트수, 음성판정자, 확진자 날짜별 데이터

In [ ]:
daily_report =pd.read_csv('../input/coronavirusdataset/time.csv')
daily_report.head(5)

In [ ]:
col = ['confirmed','negative','test']
dataset = daily_report[col]
print(dataset.info())

import datetime
end = datetime.datetime.now() - datetime.timedelta(1)
date_index = pd.date_range('2020-01-20', end)
dataset.index = date_index

fig = px.area(dataset, x=dataset.index, y='test' )
fig.show()

In [ ]:
dataset.tail()

In [ ]:
import cufflinks as cf 
cf.go_offline(connected=True)
dataset.iplot( fill=True)

# 한국 데이터셋을 가지고 분석하기에 앞서 전세계적으로 변화하는 수치를 가지고 시각화하였습니다

# before EDA analysis of Korea nCov-19 , I checked the data of international Confirmed count
by https://www.worldometers.info/coronavirus/

In [ ]:
fatality_rate = pd.DataFrame({'Confirmed':[80814, 15113, 10075, 7979, 3146, 2876, 2745, 1762],
                              'Death':     [3177,   1016,  429,  67,   86,   61,    6,  41]}) 
fatality_rate = fatality_rate.rename(index={0:"China",1:"Italy", 2:"Iran", 3:"Korea", 4:"Spain", 5: "France", 6:" Germany" , 7:"USA" })
fatality_rate['fatality'] = fatality_rate['Death']  /fatality_rate['Confirmed'] *100
fatality_rate

In [ ]:
plt.rcParams['figure.figsize']=15,7
ax= fatality_rate['Confirmed'].plot(kind='bar',  title = "International nCOV-19 Confirmed patients", color=['slateblue'])
for p in ax.patches:
  left, bottom, width, height = p.get_bbox().bounds
  ax.annotate("%.f"%(height), (left+width/2, height*1.01), ha='center')
    
plt.sca(ax)
plt.box(False) # 그래프 외곽선 지우기
plt.show()

전세계적으로 2번째 높은 확진자수를 기록하다가 몇일전 3위로,, 이제 4위로 떨어졌습니다.
이란과 이탈리아가 많이 걱정되는 추세입니다.

# Display of Fatality rate 

# 그다음으로는 사망자율에 대한 치사율 그래프입니다.

In [ ]:
plt.rcParams['figure.figsize']=18,7
ax = fatality_rate['fatality'].plot(kind='barh', stacked=True, title= "International nCOV-19 fatality_rate", rot=0, color=['red', 'skyblue','darkgreen', 'orange','lightgreen','slateblue','yellow', 'darkblue']) 
for p in ax.patches: 
  left, bottom, width, height = p.get_bbox().bounds 
  ax.annotate("%.2f%%"%(width), xy=(left+width*1.05, bottom+height/2), ha='center', va='center') 

plt.sca(ax)
plt.box(False) 
plt.show()


# load Korea nCov-19

In [ ]:
df_patient = pd.read_csv('../input/coronavirusdataset/patient.csv')
df_patient

In [ ]:
df_patient.info() # most data have missing value...

id, confirmed_date ,state , Country = no missing data 결측치가 없는 데이터이고

Most of data are missing value 
그외에는 5%밖에 안되는 데이터들로 구성되어있어 사실 EDA 를 할만한 자료가 적습니다.

성별이나 연령대별 지역별 데이터는 질병관리본부 자료를 토대로 시각화하여, 현재 데이터셋과 비교하였습니다.

세부적인 info가 업데이트 되게 되면, EDA 에 매우 도움될것으로 생각합니다.

결측치가 없는 confirmed_date로만 new dataset 을 만들었습니다.

In [ ]:
# get daily confirmed count 
# 하루하루 확진자로 판명받는 데이터의 id 갯수를 세면 = 당일 확진자수가 나옵니다
daily_count = df_patient.groupby('confirmed_date').patient_id.count()
 
# get accumulated confirmed count
# 당일 확진자수를 모두 cumsum 합쳐서 누적확진자수를 구합니다.

accumulated_count = daily_count.cumsum()

plt.figure(figsize=(18,5))

color = 'tab:red'
ax = accumulated_count.plot(title='Korea nCOV19 Confirmed individuals', rot=90, color=color)
ax.set_ylabel('Accumulated_count', color=color)
plt.box(False)

# double-y axis graph = 하나의 그래프에 2개의 y축 사용
ax2 = ax.twinx()
color = 'tab:blue'
ax2 = daily_count.plot(kind='bar',color=color, alpha=0.1)
for p in ax2.patches:
  left, bottom, width, height = p.get_bbox().bounds
  ax2.annotate("%.f"%(height), (left+width/2, height*1.01), ha='center') # ha =horizontal align
ax2.set_ylabel('Confirmed_count', color=color)
ax2.tick_params(axis='y', color=color)

plt.box(False)
plt.show()

# recent data

![](http://www.cdc.go.kr/cdc/img/main/h1_logo.png)

by http://ncov.mohw.go.kr/tcmBoardView.do?brdId=&brdGubun=&dataGubun=&ncvContSeq=353378&contSeq=353378&board_id=&gubun=ALL


In [ ]:
daily_count = pd.DataFrame(daily_count)
daily_count_data =daily_count.reset_index()
daily_count_data

In [ ]:
# get accumulated confirmed count
accumulated_count = daily_count.cumsum()
accumulated_count.reset_index(inplace=True)
accumulated_count

# display by plotly module. 
# You can look for detail data when mouse cursor is put on line graph.
## plotly 시각화를 이용하면 실시간으로 그래프위에 데이터를 확인할수 있습니다
## 선위에 마우스커서를 올리면 그위치의 정보를 보여줍니다. (추가, 원하는 간격으로 확장가능)

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=daily_count_data.confirmed_date, y=daily_count.patient_id,
        name='nCOV-19 in Korea'))
# Add figure title
fig.update_layout(title_text="daily Confirmed_patient of nCOV-19 in Korea")

# Set x-axis title
fig.update_xaxes(title_text="2020 01 ~03")

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=accumulated_count.confirmed_date, y=accumulated_count.patient_id, 
               name='nCOV-19 in Korea'))

# Add figure title
fig.update_layout(title_text="Accumulated Confirmed_patient of nCOV-19 in Korea")

# Set x-axis title
fig.update_xaxes(title_text="2020 01 ~03")

# **display in double_y graph** plotly 

### 누적확진자수 선그래프와 당일 발표되는 확진자수선그래프를 하나로 표시해서
### 최고치를 찍은 확진자수를 한번에 시각화하도록 하였습니다.

누적확진자수 기울기가 가파를때 당일 확진자수가 최고치를 찍었음을 볼수있게 도와줍니다.

In [ ]:
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])


# Add traces
fig.add_trace(
    go.Scatter(x=daily_count_data.confirmed_date, y=daily_count.patient_id,
        name='Confirmed in Korea'), secondary_y=False,)

fig.add_trace(
    go.Scatter(x=accumulated_count.confirmed_date, y=accumulated_count.patient_id,
        name='Accumulated in Korea'), secondary_y=True,)

# Add figure title
fig.update_layout(title_text="Corona19 in Korea")

# Set x-axis title
fig.update_xaxes(title_text="2020 01 ~03")

# Set y-axes titles
fig.update_yaxes(title_text="<b>Confirmed</b> individual", secondary_y=False)
fig.update_yaxes(title_text="<b>Accumulated</b> individual", secondary_y=True)

fig.update_layout(
    legend=dict(x=0, y=1,
        traceorder="normal",
        font=dict(
            family="sans-serif",
            size=14,
            color="black" ),
        bgcolor="white",
        bordercolor="Black",
        borderwidth=2 ))

fig.show()

# Prophet prediction

# 시계열분석을 위해서 Prophet 모델을 이용합니다.
https://facebook.github.io/prophet/docs/trend_changepoints.html
Prophet 모델은 반드시 2개의 열 : index, y값을 가지고 있어야합니다.

In [ ]:
df_prophet = accumulated_count.rename(columns={ 'confirmed_date': 'ds', 'patient_id': 'y' })

df_prophet

# Prophet code 에서 changepoint_range=default :0.8 입니다.
2월말부터 3월까지 0.98 - 0.9 범위안에서 계속 조정해보면서 0.95 가 최적값임을 찾았습니다. 
처음부터 최적값을 찾고자 한다면 parameter 를 dictionary 로 만들어서 최적의 값을 찾아가는 방법을 이용할 수 있습니다.

In [ ]:
m = Prophet(
    changepoint_prior_scale=0.2, # increasing it will make the trend more flexible
    changepoint_range=0.95, # place potential changepoints in the first 98% of the time series
    yearly_seasonality=False,
    weekly_seasonality=False,
    daily_seasonality=True,
    seasonality_mode='additive'
)

m.fit(df_prophet)

future = m.make_future_dataframe(periods=7)
forecast = m.predict(future)


forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(7)

In [ ]:
fig = plot_plotly(m, forecast)
py.iplot(fig) 

fig = m.plot(forecast)
a = add_changepoints_to_plot(fig.gca(), m, forecast)


# Prophet은 다음 순서로 시계열에 대한 선형회귀분석 모형을 만듭니다.

*  시간 데이터의 각종 특징을 임베딩해서 계절성 추정을 한다.
*  나머지 데이터는 구간별 선형회귀(piecewise linear regression) 분석을 한다.

선형 회귀분석은 전체 시계열의 앞 80%부분을 25개의 구간으로 나누어 실시합니다.

default 값이 아닌 코드에서 changepoint 를 수정해주었기 때문에 위 그래프와 같이 13군데에서 구간을 나눠서 분석을 진행합니다.

(구간 구분점(change point)는 changepoints 속성에 있고 , 위 코드에서 하이퍼파라미터로 조정해주었습니다.)

# 선형회귀모형의 기울기와 y-절편은 param 속성의 delta와 beta값에 있다.
아래 그래프에서 기울기에 대한 구간을 그려보았습니다.
*  prophet은 마지막 20% 데이터에 대해 선형회귀분석을 하고 신뢰구간을 계산할 수 있다.
*  신뢰구간은 구간별 기울기의 분포를 라플라스 확률분포로 추정한 후 기울기 변화를 가정하는 방법으로 신뢰구간을 계산한다.


신뢰구간에 대한 자세한 내용을 아래 출처를 참고하시길 바랍니다
https://datascienceschool.net/view-notebook/8903aa20770746e78fb5b1834ab5334b/

In [ ]:
sns.distplot(m.params["delta"], kde=False, fit=sp.stats.laplace)
plt.box(False)


# Prediction line + daily Confirmed_count bar graph
기존의 데이터가 발표되는 확진자수를 같이 시각화 하였습니다.

In [ ]:
day_plus = pd.DataFrame(index=['2020-03-12', '2020-03-13','2020-03-14' ]) 
daily_count_= pd.concat([daily_count, day_plus],sort=True)
  
accumulated_count_ = daily_count_.cumsum()
accumulated_count_

# Daily Confirmed patient, and Accumulated patient EDA
* Blue line : real accumulated line
* Red line : prediction line 
* bar graphs is daily Confirmed_count.

In [ ]:
plt.figure(figsize=(25,8))
color = 'tab:red'
forecast.yhat.plot(color=color)
plt.ylabel('Accumulated_Confirmed', color=color)
plt.xlabel('days')
plt.title('Prediction by Prophet', color=color, fontdict ={'fontsize':15})
plt.ylim=[0,14000,2000]
plt.box(False)

accumulated_count_['patient_id'].plot()


daily_count_['patient_id'].plot(kind='bar', color=['coral'] )
plt.text(x=19, y=200, s='169',color='red', horizontalalignment='center') ;plt.text(x=20, y=300, s='231',color='red' ,horizontalalignment='center') 
plt.text(x=21, y=200, s='143',color='red',horizontalalignment='center') ;plt.text(x=22, y=330, s='285',color='red',horizontalalignment='center') 
plt.text(x=23, y=560, s='505',color='red',horizontalalignment='center') ;plt.text(x=24, y=610, s='571',color='red',horizontalalignment='center') 
plt.text(x=25, y=900, s='813',color='red',horizontalalignment='center') ;plt.text(x=26, y=1100, s='1061',color='red',horizontalalignment='center')
plt.text(x=27, y=650, s='600',color='red',horizontalalignment='center') ;plt.text(x=28, y=600, s='516',color='red',horizontalalignment='center')
plt.text(x=29, y=500, s='438',color='red',horizontalalignment='center') ;plt.text(x=30, y=600, s='518',color='red',horizontalalignment='center')
plt.text(x=31, y=500, s='483',color='red',horizontalalignment='center') ;plt.text(x=32, y=400, s='367',color='red',horizontalalignment='center')
plt.text(x=33, y=300, s='248',color='red',horizontalalignment='center') ;plt.text(x=34, y=200, s='131',color='red',horizontalalignment='center')
plt.text(x=35, y=300, s='242',color='red',horizontalalignment='center') ;plt.text(x=36, y=200, s='114',color='red',horizontalalignment='center')

plt.annotate('',xy=(14,10), xytext=(14,700), arrowprops=dict(facecolor='black'))
plt.text(x=13, y=800, s='secodary infection by 31 patient',color='green', fontweight='bold')
plt.annotate('',xy=(19,1000), xytext=(19,3300), arrowprops=dict(facecolor='black'))
plt.text(x=19, y=3500, s='president\'s Declaration',color='blue',fontweight='bold')


plt.show()

# 완치자 EDA - Released_count
### groupby('released date') 완치자들의 dataset 을 뽑아냈습니다.
Dataset 의 업데이트중에 최근 data는 추가해주었습니다. 국내에 보고된 수치와 많이 달라서 Dataset 의 정확성이 떨어지고 있습니다.
03-05 발표로 보면 격리해제가 총 88명으로.. 격리해제 완치자수가 40명이상 급증했습니다.
데이터셋에서는 3명으로 되어있으니... 심각하게 차이나네요.

![](http://ncov.mohw.go.kr/front_new/modules/img_view.jsp?img_loc=/upload/mwEditor/202003/1583404684656_20200305193804.png)

# 03-05 on dataset is mismatched compared to Korea CDC report
## 업데이트되는 데이터셋에 자동으로 완치자수가 합쳐지는데,, 3월 5일 완치자수가 3명으로 되어있습니다.
# need to change value on dataset 데이터셋의 수정이 필요한거같네요. 

# 잘못된 데이터 3월5일과 3월7일의 데이터는 삭제했고, 올바른 데이터를 추가합니다.

In [ ]:
daily_released_count = df_patient.groupby('released_date').patient_id.count()
daily_released_count = pd.DataFrame(daily_released_count)

daily_released_count = daily_released_count.drop(['2020-03-05','2020-03-07'])

daily_plus = pd.DataFrame([47,20,10,12,36,81,41,45,177],
                          index=['2020-03-05','2020-03-06','2020-03-07','2020-03-08','2020-03-09','2020-03-10', '2020-03-11','2020-03-12','2020-03-13'],
                          columns=['patient_id'])
daily_released_count =pd.concat( [daily_released_count ,daily_plus] )
daily_released_count

# 3-5 완치자수(격리해제수)가 47명으로 많은 환자분의 치료가 완료되었습니다.
추후의 치료기간을 계산해서 회복율의 수치를 계산하면 전염병모델인 SIR 모델의 parameter 로 사용될수 있습니다

In [ ]:
accumulated_released_count = daily_released_count.cumsum()
accumulated_released_count.tail()

In [ ]:
plt.rcParams['figure.figsize']=10,6
color = 'tab:blue'
ax = daily_released_count.plot(kind='bar', title = 'Korea nCOV-19 Recovered individu1es', color=color, alpha=0.1, legend=False)
for p in ax.patches:
  left, bottom, width, height = p.get_bbox().bounds
  ax.annotate("%.f"%(height), (left+width/2, height*1.01), ha='center') 
ax.set_ylabel('Recovered_count', color=color)
ax.tick_params(axis='y', color=color)
ax.hlines(y = 10, xmin=0, xmax=28, colors=color, linestyles='dotted')
ax.hlines(y = 40, xmin=0, xmax=28, colors=color, linestyles='dotted')
plt.box(False)


color='tab:green'
ax2= accumulated_released_count.plot( color=color, legend=False)
ax2.set_ylabel('Accumulated_count', color=color)
ax2.tick_params(axis='y', color=color)
ax2.hlines(y = 50, xmin=0, xmax=28, colors='green', alpha=0.4, linestyles='dotted')
ax2.hlines(y = 100, xmin=0, xmax=28, colors='green', alpha=0.4,  linestyles='dotted')
ax2.hlines(y = 150, xmin=0, xmax=28, colors='green', alpha=0.4,  linestyles='dotted')
ax2.hlines(y = 200, xmin=0, xmax=28, colors='green', alpha=0.4,  linestyles='dotted')


## Accumulated Released individules count was doubled on 2020-03-05 compared to 2020-03-04
03-05 하루의 완치자수는 47명의 데이터는 추후 EDA 를 통한 회복율이 조금 달라질수 있습니다.

## 2020-03-13, 177 patients ware recoveried ! congratulation !
기존에 확진자가 많았던 만큼, 2주의 평균기간을 회복기로 보고, 많은 환자들이 앞으로도 격리해제 될것으로 보여집니다

In [ ]:
data = df_patient[["patient_id","sex","birth_year","confirmed_date","released_date","deceased_date","state"]]
released_base_data = data.dropna(subset=['released_date'])
released_base_data = released_base_data.drop(['deceased_date'], axis=1)
released_base_data.info() # 03-04 data / not updated on 03-05 / 47 peoples data 

# 격리해제 데이터가 55명으로 일주일째 멈춰있습니다.
# No update on released date. I can't analize about recovery

In [ ]:
released_base_data.confirmed_date = pd.to_datetime(df_patient.confirmed_date)
released_base_data.released_date = pd.to_datetime(df_patient.released_date)

# recovered_period =치료기간(회복기간) 계산하기
released_base_data['recovered_period'] = released_base_data['released_date'].dt.date - released_base_data['confirmed_date'].dt.date
released_base_data['recovered_period'] = released_base_data['recovered_period'].dt.days.astype(int)
# Age
released_base_data["age"] = released_base_data["confirmed_date"].dt.year - released_base_data["birth_year"]
released_base_data = released_base_data.drop(['birth_year'], axis=1)
# Age_class
released_base_data["age_class"] = pd.cut(released_base_data["age"], np.arange(0, 100, 10), include_lowest=True, right=False)
released_base_data

In [ ]:
boxplot = pd.DataFrame(released_base_data['recovered_period'].describe()).T
boxplot

In [ ]:
red_square = dict(markerfacecolor='r', marker='s')
fig, ax1 = plt.subplots(figsize=(8,2))
ax1.set_title('Recovered time on Korea nCOV-19 patient')
ax1.set_ylabel('Period')
ax1.boxplot(boxplot,  vert=False, flierprops=red_square)

# Classification of age_generation

In [ ]:
released_period= released_base_data.pivot_table( index="state", columns='age_class', values='recovered_period', aggfunc='count').fillna('0')
released_period = released_period.T
released_period.reset_index(inplace=True)
released_period

In [ ]:
released_period.index=['10-19','20-29','30-39','40-49','50-59','60-69', '70-']

import plotly.express as px

fig = go.Figure(data=[
    go.Bar(name='All', x=released_period.index, y=released_period.released)])


fig.show()

# 완치자수를 연령대별 몇명씩 분포되었는지 확인후, 성별로 재확인하였습니다.

# Classification of sex

In [ ]:
released_by_age= released_base_data.pivot_table( index="sex", columns='age_class', values='released_date', aggfunc='count').fillna('0')
released_by_age = released_by_age.T
released_by_age.reset_index(inplace=True)
released_by_age

In [ ]:
released_by_age.index=['10-19','20-29','30-39','40-49','50-59','60-69', '70-']


fig = go.Figure(data=[
    go.Bar(name='Female', x=released_by_age.index, y=released_by_age.female),
    go.Bar(name='Male', x=released_by_age.index, y=released_by_age.male)])

fig.update_layout(barmode='group')
fig.show()

# 질병관리본부 정례브리핑 발표자료에 의한 EDA

http://ncov.mohw.go.kr/tcmBoardView.do?brdId=&brdGubun=&dataGubun=&ncvContSeq=353378&contSeq=353378&board_id=&gubun=ALL

[보도참고자료] 코로나바이러스감염증-19 국내 발생 현황(정례브리핑).hwp 다운받아보세요

# 3월 11일 기준 국내 확진자수 성별 분포

In [ ]:
Confirmed_sex = pd.DataFrame({'sex':[4661,2852], 'rate':[62.0,38.0]})
Confirmed_sex = Confirmed_sex.rename(index={0:"female", 1:"male"})
Confirmed_sex

In [ ]:

plt.rcParams['figure.figsize']=12,8
group_explodes = (0.05, 0) #explode 1st slice
plt.pie(Confirmed_sex['sex'], explode=group_explodes,labels=["female","male"], colors=['lightcoral', 'lightskyblue'],autopct='%1.2f%%', shadow=True, startangle=90, textprops={'fontsize':14})
plt.axis('equal')
plt.title('Confirmed individual by sex')
plt.show()

# 3월 11일 기준 국내 확진자수 연령대별 분포

In [ ]:
Confirmed_age = pd.DataFrame({'Confirmed':[67,393,2213,789,1030,1416,926,454,222], 'rate':[0.9, 5.2, 29.5, 10.5, 13.7, 18.8, 12.4, 6.0, 3.0]})
Confirmed_age = Confirmed_age.rename(index={0:"0-9", 1:"10-19", 2:"20-29", 3:"30-39", 4:"40-49", 5:"50-59", 6:"60-69", 7:"70-79", 8:"80-"})
Confirmed_age

In [ ]:
plt.rcParams['figure.figsize']=12,5
color = 'tab:red'
ax= Confirmed_age['Confirmed'].plot( title = " nCOV19 - Confirmed individule by Age", rot=0,color=color)
ax.set_ylabel('Confirmed_count',color=color)
ax.tick_params(axis='y',color=color)
ax.legend()

ax2 = ax.twinx()
color = 'tab:blue'
ax2 = Confirmed_age['rate'].plot(kind='bar', color=color, alpha=0.1)
ax2.set_ylabel(' %')

for p in ax2.patches: 
  left1, bottom1, width1, height1 = p.get_bbox().bounds 
  ax2.annotate("%.f%%"%(height1), xy=(left1+width1/2, bottom1+height1/2), ha='center', va='center')

plt.sca(ax)
plt.show()

# select data by infection_reason

확진자수 dataset에는 감염이유로 밝혀진 수는 146명으로 모집단인 5328 의 2%에 불과합니다. 분석용으로는 큰의미가 없습니다.
현재 Deagu 지역이나 group 인 경우 = 가장 큰 감염이유인  "contact with patient" 로 missing value 를 replace 해준다면 좀더 표본집단이 커질것이라 생각을 해봅니다.

In [ ]:
df_patient.info() # infection_reason    154 non-null object

In [ ]:
pd.DataFrame(df_patient["infection_reason"].value_counts())

In [ ]:
reason_dict = {   
    "contact with the patient": "contact with patient", 
    "visit to Shincheonji Church" : "contact with patient", 
    "contact with patient in Japan":  "visit to other Country",
    "contact with patient in Singapore":  "visit to other Country",
    "visit to Daegu" : "contact with patient",  
    "visit to Shincheonji Church" : "contact with patient",

    "visit to Wuhan" : "visit to China",
    " visit to China" :  "visit to China",
    "visit to China": "visit to China",
    "residence in Wuhan" : "residence in Wuhan",

    "visit to Thailand":  "visit to other Country",
    "visit to Japan": "visit to other Country",
    "visit to Italy":  "visit to other Country",
    "visit to Vietnam":  "visit to other Country",

    "contact with patient in Japan":  "visit to other Country",
    "visit to Cheongdo Daenam Hospital" :"contact with patient",
    "pilgrimage to Israel":  "visit to other Country",
    "visit to ooo" :"contact with patient",
    "contact with patient in Daegu" :"contact with patient"

}
infection_reason =pd.DataFrame(df_patient["infection_reason"].replace(reason_dict).value_counts())
infection_reason



In [ ]:
infection_reason.reset_index(inplace=True)

# select data by region

# 지역데이터가 밝혀진 환자들중 Daegu patient =48 이라 솔직히 무의미한 분석입니다.


In [ ]:
region = pd.DataFrame(df_patient["region"].value_counts())
region.reset_index(inplace=True)
region

### 위데이터는 수도와 경기도가 가장많은 환자로 나타나는데 
### 질병관리본부의 정례브리핑 자료에 따르면 현재 대구만 4000명을 넘었으니까요.

In [ ]:
infection_reason.index=['contact with patient','visit to other Country','visit to China','residence in Wuhan']
region.index = ['capital area','Gyeongsangbuk-do','Daegu','Daejeon','Gwangju','Gangwon-do','Jeju-do',
                'filtered at airport','Jeollabuk-do','Jeollanam-do', 'Ulsan','Chungcheongbuk-do','Busan','Chungcheongnam-do']

fig = make_subplots(rows=2, cols=1)

fig.add_trace(go.Bar(x=infection_reason.index, y=infection_reason.infection_reason,
                    marker=dict(color=infection_reason.infection_reason, coloraxis="coloraxis")),  1, 1)


fig.add_trace(go.Bar(x=region.index, y=region.region,
                    marker=dict(color=region.region, coloraxis="coloraxis")),  2, 1)

fig.update_layout(coloraxis=dict(colorscale='Bluered_r'), showlegend=False)
fig.update_layout(height=1000, showlegend=False)
fig.show()

# CDC 질병관리본부 발표일 3월 9일 기준 지역별 확진자수 

In [ ]:
region_by3_9 = pd.DataFrame({'patient':[5571,1107,152,130,102, 96, 83], 'rate':[75.47, 15.0, 2.06, 1.76, 1.38, 1.3, 1.12]})
region_by3_9 = region_by3_9.rename(index={0:"Deagu", 1:"Gyeongbuk", 2:"Gyeonggi", 3:"Seoul", 4:"Chungnam", 5:"Busan", 6:"Gyeongnam"})
region_by3_9

In [ ]:
plt.rcParams['figure.figsize']=15,4
color = 'tab:red'
ax= region_by3_9['patient'].plot( title = " nCOV19 - Confirmed individule by region", rot=0,color=color)
ax.set_ylabel('Confirmed_count',color=color)
ax.tick_params(axis='y',color=color)
ax.legend()

ax2 = ax.twinx()
color = 'tab:blue'
ax2 = region_by3_9['rate'].plot(kind='bar', color=color, alpha=0.1)
ax2.set_ylabel(' %')

for p in ax2.patches: 
  left1, bottom1, width1, height1 = p.get_bbox().bounds 
  ax2.annotate("%.f%%"%(height1), xy=(left1+width1/2, bottom1+height1/2), ha='center', va='center')

plt.sca(ax)
plt.show()

# display pie chart

In [ ]:
region_by3_9.reset_index(inplace=True)
region_by3_9.index=["Deagu","Gyeongbuk","Gyeonggi","Seoul","Busan","Chungnam","Gyeongnam"]
fig = px.pie(region_by3_9, values='rate', names='index')
fig.show()

# 질병관리본부에 보고된 정례브리핑에 따른 지역별 분포 파이차트와
# 캐글데이터셋 case 에서 뽑아온 지역별 분포 파이차트가 유사하게 나타납니다.

In [ ]:
count.reset_index(inplace=True)
count.index=["Busan","Chungcheongbuk-do","Chungcheongnam-do","Daegu","Daejeon","Gangwon-do","Gwangju","Gyeonggi-do","Gyeongsangbuk-do","Gyeongsangnam-do",
            "Incheon","Jeju-do","Jeollabuk-do","Jeollanam-do","Sejong","Seoul","Ulsan"]
fig = px.pie(count, values='confirmed', names='index')
fig.show()

# classification of sex

id                  7383 non-null int64

sex                almost - null value

In [ ]:
sex = pd.DataFrame(df_patient["sex"].value_counts())
sex = pd.DataFrame(df_patient["sex"].replace({'feamle':'female'}).value_counts())
sex

In [ ]:
sex.reset_index(inplace=True)
sex.index=['female','male']
import plotly.express as px
fig = px.pie(sex, values='sex', names='index')
fig.show()

# It doesn't matched on real data
bottom Pie graph is based on data by CDC

In [ ]:
plt.rcParams['figure.figsize']=12,8
group_explodes = (0.05, 0) #explode 1st slice
plt.pie(Confirmed_sex['sex'], explode=group_explodes,labels=["female","male"], colors=['lightcoral', 'lightskyblue'],autopct='%1.2f%%', shadow=True, startangle=90, textprops={'fontsize':14})
plt.axis('equal')
plt.title('Confirmed individual by sex')
plt.show()

# 사망자가 많아짐에 따라, 사망자에 따른 연령별 성별 EDA 를 추가함

In [ ]:
data = df_patient[["patient_id","sex","birth_year","confirmed_date","deceased_date","state"]]
death_data = data.dropna(subset=['deceased_date'])
death_data

In [ ]:
death_data.confirmed_date = pd.to_datetime(df_patient.confirmed_date)
death_data.deceased_date = pd.to_datetime(df_patient.deceased_date)

# deceased_period 사망까지 소요된 시간 계산
death_data['deceased_period'] = death_data['deceased_date'].dt.date - death_data['confirmed_date'].dt.date
# Age
death_data["age"] = death_data["confirmed_date"].dt.year - death_data["birth_year"]
death_data = death_data.drop(['birth_year'], axis=1)
# Age_class
death_data["age_class"] = pd.cut(death_data["age"], np.arange(0, 100, 10), include_lowest=True, right=False)
death_data

In [ ]:
mean = death_data['deceased_period'].dt.days.mean()
print('평균 사망까지 걸린 시간 %.2f%%'%(mean),'days')

In [ ]:
death_data = death_data.dropna(subset=['age_class'])
death_data

In [ ]:
death_data_by_age = death_data.pivot_table( index="state", columns='age_class', values='deceased_date', aggfunc='count').fillna('0')
death_data_by_age = death_data_by_age.T
death_data_by_age.reset_index(inplace=True)
death_data_by_age

In [ ]:
death_data_by_age.index=['30-39','40-49','50-59','60-69', '70-79','80-89']

import plotly.express as px

fig = go.Figure(data=[
    go.Bar(name='All', x=death_data_by_age.index, y=death_data_by_age.deceased)])


fig.show()


In [ ]:
death_data= death_data.pivot_table( index="sex", columns='age_class', values='deceased_date', aggfunc='count').fillna('0')
death_data = death_data.T
death_data.reset_index(inplace=True)
death_data

In [ ]:
death_data.index=['30-39','40-49','50-59','60-69', '70-79', '80-']

import plotly.express as px

fig = go.Figure(data=[
    go.Bar(name='Female', x=death_data.index, y=death_data.female),
    go.Bar(name='Male', x=death_data.index, y=death_data.male)])

fig.update_layout(barmode='group')
fig.show()

In [ ]:
death_count = df_patient.groupby('deceased_date').patient_id.count()
death_count = pd.DataFrame(death_count)
death_accumulated = death_count.cumsum()

In [ ]:
plt.rcParams['figure.figsize']=10,6

color='tab:blue'
ax= accumulated_released_count.plot(kind='bar' ,title = 'Korea nCOV-19 Recoved & Death individules',
                                    alpha=0.5, color=color, legend=False)
ax.set_ylabel('Recovered_count', color=color)
ax.tick_params(axis='y', color=color)
plt.box(False)
color = 'black'
ax2 = death_accumulated.plot(kind='bar', color=color, alpha=0.5, legend=False)
ax2.set_ylabel('Deceased_count', color=color)
ax2.tick_params(axis='y', color=color)
plt.box(False)



# to be continuous... Machine Learning

# updated by 2020-03-07
I tried Machine Learning on Korea-Corona-19 Confirmed count data

# SVM ( Support Vector Machine ), Linear Regression

In [ ]:
daily_count = df_patient.groupby('confirmed_date').patient_id.count()
daily_count = pd.DataFrame(daily_count)
data = daily_count.cumsum()
data 

In [ ]:
dataset = data.iloc[14:]
dataset

# Modeling 을 위해서 numpy 배열로 만든다.

In [ ]:
# Future forcasting
days_in_future = 3
dates = pd.date_range('2020-2-18','2020-3-11')
future_forcast = np.array([i for i in range(len(dates)+days_in_future)]).reshape(-1, 1)
# future_forcast #3일을 추가해서 > 미래 3일의 그래프를 그려본다.

days = np.array([i for i in range(len(dates))]).reshape(-1, 1) # index -> ndarray
data = np.array(dataset).reshape(-1, 1) # count->ndarray

In [ ]:
kernel = ['linear', 'rbf']
c = [0.01, 0.1, 1, 10, 100]
gamma = [ 0.001, 0.01, 0.1, 10]
epsilon = [0.01, 0.1, 1, 10, 100]
shrinking = [True, False]
svm_grid = {'kernel': kernel, 'C': c, 'gamma' : gamma, 'epsilon': epsilon, 'shrinking' : shrinking}

from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
svm = SVR()
svm_search = GridSearchCV(svm, svm_grid, scoring='neg_mean_squared_error', cv=5, return_train_score=True, n_jobs=-1,verbose=1)

# 모델 fitting 전에 train, test 를 나눠서 해봤으나, 현재 데이터셋이 너무 범위가 작아 확진자증가를 반영하지 못해. 전체 데이터셋으로 fitting 함.
svm_search.fit(days, data)

In [ ]:
svm_search.best_params_

In [ ]:
svm_search.best_estimator_

In [ ]:
svm_pred = svm_search.best_estimator_.predict(future_forcast)
svm_pred

# 최소자승법 OLS - Simple Linear Regression

In [ ]:
import statsmodels.api as sm

lm= sm.OLS(data,days)
results =lm.fit()
results.summary()

# LinearRegression

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error

linear_model = LinearRegression(fit_intercept=False, normalize=True)
linear_model.fit(days, data)
linear_pred = linear_model.predict(future_forcast)
linear_model.coef_

In [ ]:
plt.plot(svm_pred, color='green', ls='-.', label = 'Prediction by SVM')
plt.plot(linear_pred, color='red', ls='--', label='Prediction by Linear Regression')
plt.plot(dataset, label='Accumulated real count')
plt.xlabel('days')
plt.vlines(x=22, ymin=0, ymax=10000, alpha=0.3, linestyles='--')
plt.text(x=24, y=5000, s='prediction',color='black', fontsize =20,horizontalalignment='center') 
plt.xticks(rotation=90, ha='left')
plt.legend()
plt.box(False)

Machine Learning 으로 위의 예측모델을 만들었으나, 추가로 딥러닝 모델로 도전해보기로 했습니다.

아직 딥러닝공부를 마스터하지 않았기때문에 틀린부분에 대해 첨삭 부탁드립니다.

선형회귀로 시계열데이터를 반영한 시계열분석과 MLP, LSTM 을 시도했습니다.

# Deep Learning  for Analysis

1. 시계열 분석 ARIMA 모델 ( LSTM 과 비교하기 위함)
2. MLP 멀티퍼셉트론
3. LSTM Neural Network
케라스 TimeseriesGenerator 으로 예측하기



![대체 텍스트](https://miro.medium.com/max/696/0*kY5G6-jVm6iII65m.png)

In [ ]:
dataset.columns = ['Confirmed']
len(dataset)

In [ ]:
data = np.array(dataset).reshape(-1, 1) # count->ndarray

In [ ]:
train_data = dataset[:len(dataset)-3]
test_data = dataset[len(dataset)-3:]

* # Prophet model 로 예측한 값을 가장 마지막에 ARIMA 와 비교해보려고 합니다.
# ARIMA model
1. 시계열 자료 - 시간의 흐름에 따라 관찰된 데이터


시계열 모형
*  (1) 자기회귀 모형 (Autoregressive model, AR)
*  (2) 이동평균 모형 (Moving average model, MA)
*  (3) AR + MA = ARMA

ARIMA, 자기회귀누적이동평균 모형 (Autoregressive integrated moving average model)
- 비정상 시계열 모형 차분이나 변환을 통해 AR, MA, 또는 이 둘을 합한 ARMA 모형으로 정상화 
-    ARIMA(p, d, q) - d : 차분 차수 / p : AR 모형 차수 / q : MA 모형 차수


모형 그 자체에 이러한 비정상성을 제거하는 과정을 포함한것이 ARIMA모형이며 ARIMA(p, d, q)로 표현한다.
 
      I: Integrated. 누적을 의미하는 것으로, 차분을 이용하는 시계열모형들에 붙이는 표현이라고 생각하면 편하다. 
      차분(Diffrence)횟수를 의미하는 d



In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
#시계열 자료 {  Yt } 의 상관 함수는 acf, pacf, iacf 가 있는데 이는 ARMA 모형 진단에 사용된다.
diff_1=train_data.diff(periods=1).iloc[1:] # diff() 차분함수 => 1차 차분함수
diff_1.plot()
plot_acf(diff_1)
plot_pacf(diff_1)

! pip install pmdarima # 설치해서 써야함.

## from pmdarima import auto_arima
## auto_arima(accumulated_count, seasonal=False)

한달간 일어난 전염병 시계열데이터로 계절성 요인은 False 로 지정한다.

Auto_arima() 함수를 이용해서 p,d,q=() 알아낼수 있다. 

위의 ACF, PCAF 를 참고해서 (1,2,1)로 대입했을때 가장 적은 AIC =215, BIC =217로 설명됨을 확인했다


In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

arima_model = SARIMAX(train_data['Confirmed'].values, order = (1,2,1))
arima_result = arima_model.fit(trend='c', full_output=True, disp=True)
arima_result.summary()

In [ ]:
pred_Arima = arima_result.forecast(steps=3)
pred_Arima

# MLP 멀티퍼셉트론

In [ ]:
x = days[:20] # train test split
y = train_data.values
test = days[20:]

In [ ]:
from sklearn.neural_network import MLPRegressor
model = MLPRegressor(hidden_layer_sizes=[20, 10, 10, 3], max_iter=20000, random_state=42)
MLP = model.fit(x, y)

In [ ]:
pred_MLP = model.predict(test)
pred_MLP

# LSTM with Time Series Generator

In [ ]:
# Before creating LSTM model we should create a Time Series Generator object.

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(train_data)
scaled_train_data = scaler.transform(train_data)
scaled_test_data = scaler.transform(test_data)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
n_input =2
n_features =1
                             
generator = TimeseriesGenerator(scaled_train_data,scaled_train_data, length=n_input, batch_size=1)

lstm_model = Sequential()
lstm_model.add(LSTM(19, activation='relu', input_shape = (n_input, n_features)))
lstm_model.add(Dense(10))
lstm_model.add(Dense(5))
lstm_model.add(Dense(1))
lstm_model.compile(optimizer='adam', loss='mse')
lstm_model.summary()


In [ ]:
lstm_model.fit_generator(generator, epochs=20)

In [ ]:
lstm_model.history.history.keys()

In [ ]:
losses_lstm = lstm_model.history.history['loss']
plt.figure(figsize = (12,4))
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.xticks(np.arange(0,21,1))
plt.plot(range(len(losses_lstm)), losses_lstm)

In [ ]:
lstm_predictions_scaled = []

batch = scaled_train_data[-n_input:]
current_batch = batch.reshape((1, n_input, n_features))

for i in range(len(test_data)):   
    lstm_pred = lstm_model.predict(current_batch)[0]
    lstm_predictions_scaled.append(lstm_pred) 
    current_batch = np.append(current_batch[:,1:,:],[[lstm_pred]],axis=1)


# As you know we scaled our data that’s why we have to inverse it to see true predictions.
lstm_predictions_scaled

In [ ]:
lstm_predictions = scaler.inverse_transform(lstm_predictions_scaled) # 열로 넣어줘야 시각화 할수 있음
lstm_predictions

In [ ]:
test_data['ARIMA'] = pred_Arima
test_data['MLP regression'] = pred_MLP
test_data['LSTM_Predictions'] = lstm_predictions
test_data

In [ ]:
test_data.plot()

In [ ]:
print('MAE of ARIMA Model ', mean_absolute_error(test_data['Confirmed'], test_data['ARIMA']))
print('MAE of MLP Model ',mean_absolute_error(test_data['Confirmed'], test_data['MLP regression']))
print('MAE of LSTM Model ',mean_absolute_error(test_data['Confirmed'], test_data['LSTM_Predictions']))
print('----------------------------------------------------')
print('MSE of ARIMA Model ', mean_squared_error(test_data['Confirmed'], test_data['ARIMA']))
print('MSE of MLP Model ',mean_squared_error(test_data['Confirmed'], test_data['MLP regression']))
print('MSE of LSTM Model ',mean_squared_error(test_data['Confirmed'], test_data['LSTM_Predictions']))

In [ ]:
from sklearn.metrics import r2_score
print('Coefficient of determination : r2_score of ARIMA Model ', r2_score(test_data['Confirmed'], test_data['ARIMA']))
print('Coefficient of determination : r2_score of MLP Model ',r2_score(test_data['Confirmed'], test_data['MLP regression']))
print('Coefficient of determination : r2_score of LSTM Model ',r2_score(test_data['Confirmed'], test_data['LSTM_Predictions']))

# 결정계수가 - 음수로 나온다는건, 모델이 의미가 없다는걸로 해석되네요.. 
# 시계열분석인 ARIMA 만 설명력이 양수로 0.50 으로 확인됩니다.. (비계절성 ARIMA 모형입니다.)

# Prophet 모델과 Arima model 예측값 비교

prophet은 페이스북에서 개발한 시계열 예측 패키지다. ARIMA와 같은 확률론적이고 이론적인 모형이 아니라 몇가지 경험적 규칙(heuristic rule)을 사용하는 단순 회귀모형이지만 단기적 예측에서는 큰 문제 없이 사용할 수 있다.

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
data = np.array(dataset).reshape(-1, 1)
arima_model = SARIMAX(data, order = (1,2,1))
arima_result = arima_model.fit(trend='c', full_output=True, disp=True)
arima_result.summary()

In [ ]:
pred_Arima = arima_result.forecast(steps=6) 
pred_Arima = pd.DataFrame(pred_Arima, columns=['predict'])
pred_Arima

In [ ]:
forecast[['ds', 'yhat']].tail(7)